#Project 3: Wrangling OpenStreetMap Data

<b>Author:</b> Nicholas Hunt-Walker

<b>Start-date:</b> November 11, 2015

## First data description
<ul>
<li><b>Location:</b> Austin, TX</li>
<li><b>Data Retrieved:</b> November 10, 2015</li>
<li><b>Retrieved From:</b> [MapZen](https://mapzen.com/data/metro-extracts)</li>
<li><b>XML File Size:</b> 92.3 MB</li>
</ul>

In [1]:
import xml.etree.cElementTree as ET
from pymongo import MongoClient
from datetime import datetime
import re

## Step 1: Traverse the XML Tree

In [2]:
node_types = set()
node_ids = set()
node_count = {}
data = []

infile = "austin_texas.osm"
# infile = "auburn_alabama.osm"

CREATED = ["version", "changeset", "timestamp", "user", "uid"]
POSITION = ["lat", "lon"]
format_gnis = {
    "gnis:Class" : "feature_class", "gnis:County" : "county",
    "gnis:County_num" : "county_fips_code", "gnis:ST_alpha" : "state_abbrv",
    "gnis:ST_num" : "state_fips_code", "gnis:county_id" : "county_fips_code",
    "gnis:created" : "entry_date", "gnis:state_id" : "state_fips_code",
    "gnis:id" : "gnis_id"
}
gnis_multiples = ["feature_id", "entry_date"]
tiger_multiples = ["tlid", "county", "cfcc", "name_base"]

format_address = {
    "addr:housenumber" : "house_number", "addr:housename" : "house_name",
    "addr:street" : "street_address", "addr:postcode" : "post_code",
}

street_mapping = { 
            "St": "Street", "Ave": "Avenue", "Rd": "Road", 
            # I added the following
            "Bld" : "Building", "Bldg" : "Building", "Blvd" : "Boulevard",  
            "Cir" : "Circle", "Ct" : "Court", "Cv" : "Cove",              
            "Dr" : "Drive", "Expwy" : "Expressway", "Hwy" : "Highway",              
            "HWY" : "Highway", "Ln" : "Lane", "Pkwy" : "Parkway",  
            "Ste" : "Suite", "STE" : "Suite", "Wy" : "Way",              
            }
lower_these = ["amenity", "cuisine"]

def clean_input(the_str, sep=None):
    cleaned_str = the_str.replace(u"\u2013", "-").replace("en:", "")
    if sep:
        cleaned_str = cleaned_str.split(sep)
    
    return cleaned_str

def correct_street_address(street_name):
    for key in street_mapping.keys():
        m = re.search("(" + key + ")(?![a-z])", street_name)
        if m:# and (street_name.find(street_mapping[key]) == -1):
            street_name = street_name.replace(key, street_mapping[key])
    
    street_name = street_name.replace(".","").replace("#","")
    return street_name          

def correct_post_code(code):
    search_str = re.compile(r'^[A-Z]+', re.IGNORECASE)
    anomaly = re.search(search_str, code)
    if anomaly:
        code = code.replace(anomaly.group(), "").strip().split(u"\u2013")[0][:5]
    else:
        return code[:5]
    
    if code == "":
        return "Invalid"
    else:
        return code[:5]

for _, element in ET.iterparse(infile):
    the_tag = element.tag
    
    if the_tag in node_types:
        node_count[the_tag] += 1
    else:
        node_types.add(the_tag)
        node_count[the_tag] = 1

    if (the_tag == "way") or (the_tag == "node"):
        new_node_way = {"type":the_tag,
                        "created":{}}

        for key in element.attrib.keys():
            if key in CREATED:
                new_node_way["created"][key] = element.attrib[key]
            elif key not in POSITION:
                new_node_way[key] = element.attrib[key]
                
        if the_tag == "node":
            new_node_way["position"] = []
            if len(new_node_way["position"]) < 2:
                new_node_way["position"].append(float(element.attrib["lat"]))
                new_node_way["position"].append(float(element.attrib["lon"]))

        if the_tag == "way":
            if "nd" in [em.tag for em in element.getchildren()]:
                new_node_way["node_refs"] = []
                for tag in element.iter("nd"):
                    new_node_way["node_refs"].append(tag.attrib["ref"])

        if "tag" in [em.tag for em in element.getchildren()]:
            for tag in element.iter("tag"):
                if tag.attrib["k"].startswith("tiger"):
                    if "tiger_tags" not in new_node_way.keys():
                        new_node_way["tiger_tags"] = {}
                        
                    new_key = tag.attrib["k"].split(":")[1]
                    if new_key in tiger_multiples:
                        new_input = clean_input(tag.attrib["v"], sep=":")
                    else:
                        new_input = clean_input(tag.attrib["v"])
                    new_node_way["tiger_tags"][new_key] = new_input
                    
                elif tag.attrib["k"].startswith("addr"):
                    if "address" not in new_node_way.keys():
                        new_node_way["address"] = {}
                    
                    if tag.attrib["k"] in format_address.keys():
                        new_key = format_address[tag.attrib["k"]]
                        
                    elif tag.attrib["k"].split(":")[1] != "state":
                        new_key = tag.attrib["k"].split(":")[1]
                        
                    new_input = clean_input(tag.attrib["v"])
                    if new_key == "city":
                        new_input = new_input.split(',')[0]
                        if new_input.lower() == "tx" or new_input.lower() == "texas":
                            new_input = "Austin"
                        
                    if new_key == "street_address":
                        new_input = correct_street_address(new_input)
#                         street_addresses.add(new_input)
                        
                    if new_key == "post_code":
                        new_input = correct_post_code(new_input)
                        
                    new_node_way["address"][new_key] = new_input
                    
                elif tag.attrib["k"].startswith("gnis"):
                    if "gnis" not in new_node_way.keys():
                        new_node_way["gnis"] = {}
                        
                    if tag.attrib["k"] in format_gnis.keys():
                        new_key = format_gnis[tag.attrib["k"]]
                    else:
                        new_key = tag.attrib["k"].split(":")[1]
                        
                    if new_key in gnis_multiples:
                        new_input = clean_input(tag.attrib["v"], sep=";")
                    else:
                        new_input = clean_input(tag.attrib["v"])
                    new_node_way["gnis"][new_key] = new_input
                    
                else:
                    new_input = clean_input(tag.attrib["v"])
                    if tag.attrib["k"] == "maxspeed":
                        if new_input.endswith("mph"):
                            new_input = new_input.replace("mph", " mph").replace("  ", " ")
                            
                        else:
                            new_input = new_input + " mph"
                    if tag.attrib["k"] in lower_these:
                        new_node_way[tag.attrib["k"]] = new_input.lower()
                    else:
                        new_node_way[tag.attrib["k"]] = new_input

        data.append(new_node_way)

In [3]:
# The total numbers for each type of tag in the XML file
node_count

{'bounds': 1,
 'member': 13537,
 'nd': 951652,
 'node': 822765,
 'osm': 1,
 'relation': 1315,
 'tag': 549875,
 'way': 85505}

## Step 2: Insert the Resulting JSON into MongoDB

In [4]:
client = MongoClient()
db = client.examples
db.drop_collection("city_data")

In [5]:
for ii in range(len(data)):
    db.city_data.insert(data[ii])

Interpreted from [ScribeKey](http://www.scribekey.com/EntityAttributes/EDGES.html), and the [OSM wiki](http://wiki.openstreetmap.org/wiki/TIGER_to_OSM_Attribute_Map), these are explanations of the TIGER keys:
<ul>
<li><b>zip\_right/left: </b>ZIP code associated with the most inclusive address range on the right/left side</li>
<li><b>tiger:cfcc: </b>Census Feature Class Code. Describes the primary feature type for a linear feature. These codes are defined here [link](http://support.esri.com/fr/knowledgebase/techarticles/detail/11966). Note that these codes were replaced by the MAF/TIGER Feature Class Code MTFCC in 2007. Those class codes are defined here: [link](https://www.census.gov/geo/reference/mtfcc.html).</li>
<li><b>tiger:name\_direction\_prefix\_%n: </b>The prefix for the street name, e.g. "NW", or "SE".</li>
<li><b>tiger:name\_base\_%n: </b>The street's name alone. The regular <b>name\_base</b> is the primary name, and every other one after that is an alias</li>
<li><b>tiger:name\_type\_%n: </b>Like a "Rd", "Ave", "St", etc. </li>
<li><b>tiger:name\_direction\_suffix\_%n: </b>The suffix for the street name, e.g. "S", or "N"</li>
<li><b>name\_%n: </b>The full street name. It's a combination of "name_direction_prefix" + "name_base" + "name_type" + "name_direction_suffix"</li>
<li><b>reviewed</b> Set to "no" for ALL ways. It doesn't seem to be an accurate indication of anything in particular.</li>
</ul>

Map features are specifically described [here](http://wiki.openstreetmap.org/wiki/Map_Features)

Nodes have a different set of keys, not taken from TIGER it seems. They're taken from GNIS, along with some other descriptors.
<ul>
<li><b>GNIS tags described: </b> [here](http://wiki.openstreetmap.org/wiki/USGS_GNIS)</li>
<li><b>gnis:ST_num: </b>State FIPS code</li>
<li><b>gnis:ST_alpha: </b>State name (2-Letter abbreviation)</li>
<li><b>gnis:County_num: </b>County FIPS code</li>
<li><b>gnis:Class: </b>Feature Class name. The meaning of the feature classes are found [here](http://wiki.openstreetmap.org/wiki/USGS_GNIS)</li>
<li><b>gnis:County: </b>County name</li>
<li><b>gnis:id: </b></li>
<li><b>ele: </b>GNIS data includes elevation</li>
<li><b>place: </b>described [here](http://wiki.openstreetmap.org/wiki/Key:place)</li>
</ul>

Extra help:
- http://stackoverflow.com/questions/13093727/how-to-replace-unicode-characters-in-string-with-something-else-python
- http://regexr.com/

### Addressing issues with the data set:
- Some items with GNIS data had multiple feature_id's and entry dates, separated by semicolons. Those were captured, split by the semicolon, and put into the database as arrays.
- In the same vein, a few tags from TIGER also had multiple values, though these were separated by colons. Those were also captured and put into arrays.
- Postal codes could have a variety of configurations. I noted this specifically by testing for distinct postal codes with a set() on one of the preliminary runs of the data. Most are 5-digit format but some have A-Z characters, some have more than 5 digits, and some are entirely invalid. I used a regex string to search for any A-Z characters and remove them. If there was nothing left, I returned "Invalid". Otherwise, I took the first 5 numbers as the postal code.
- For the street addresses, I took a page out of Lesson 6 and used regex strings to search for replaceable street abbreviations. This one was particularly difficult, as sometimes an abbreviated street name could also serve as part of a word. For example "St" could be "street" or the start of "Star Avenue". Trying a simple "replace" function would turn such an address into "Streetar Avenue", so I had to search for abbreviations that were not followed by any alphabet character.
- I decided not to format the directional letters (S, N, NE, SE, SW, etc.) in the street addresses. I realized that while I could reliably replace those letter selections, I might also be wrecking the address for, say, 221 Avenue N
- Some addresses had inputs for City that were "Tx, "TX", "tx", or "Texas". I just changed all those presenting any variation of "texas" to "Austin"
- Speed limits weren't uniform so I made sure they all had "mph" at the end.

##Step 3: Get Stats from the Data

In [6]:
print "The total number of entries: {0}".format(db.city_data.find().count())
print "The total number of nodes: {0}".format(db.city_data.find({"type" : "node"}).count())
print "The total number of ways: {0}".format(db.city_data.find({"type" : "way"}).count())

The total number of entries: 908270
The total number of nodes: 822760
The total number of ways: 85497


In [7]:
number_of_unique_users = db.city_data.aggregate([
                                          {"$group" : {
                                             "_id" : "$created.user",
                                             "count" : {"$sum" : 1}
                                                      }
                                          }])
top_5_users = db.city_data.aggregate([
                                      {"$group" : {
                                            "_id" : "$created.user",
                                            "count" : {"$sum" : 1}
                                      }},
                                      {"$sort" : {"count" : -1}},
                                      {"$limit" : 5 }
                                     ])
five_most_popular_amenities = db.city_data.aggregate([{"$match" : {"amenity" : {"$exists" : 1}}},
                                                      {"$group" : {"_id" : "$amenity",
                                                                  "count" : {"$sum" : 1}}
                                                      },
                                                      {"$sort" : {"count" : -1}},
                                                      {"$limit" : 5}
                                                     ])


distinct_amenities = db.city_data.distinct("amenity")
number_of_amenities = len(distinct_amenities)
number_of_schools = int(db.city_data.find({"amenity" : "school"}).count())
number_of_cafes = int(db.city_data.find({"amenity" : "cafe"}).count())


The number of unique users

In [8]:
user_count = 0
for doc in number_of_unique_users: user_count += 1
print user_count

1021


The five most popular amenities in Austin, TX

In [9]:
for doc in five_most_popular_amenities: print doc

{u'count': 1910, u'_id': u'parking'}
{u'count': 711, u'_id': u'restaurant'}
{u'count': 591, u'_id': u'waste_basket'}
{u'count': 565, u'_id': u'school'}
{u'count': 532, u'_id': u'fast_food'}


The top 5 contributing users

In [10]:
for doc in top_5_users: print doc

{u'count': 234473, u'_id': u'woodpeck_fixbot'}
{u'count': 37679, u'_id': u'varmint'}
{u'count': 36494, u'_id': u'richlv'}
{u'count': 35579, u'_id': u'Clorox'}
{u'count': 35207, u'_id': u'Iowa Kid'}


In [11]:
print "The total number of schools: {0}".format(number_of_schools)
print "The number of cafes: {0}".format(number_of_cafes)
print "The number of distinct amenities: {0}".format(len(distinct_amenities))

The total number of schools: 565
The number of cafes: 130
The number of distinct amenities: 84


In [12]:
# amenities_counts = db.city_data.aggregate([{"$match" : {"amenity" : {"$exists" : 1}}},
#                                                       {"$group" : {"_id" : "$amenity",
#                                                                   "count" : {"$sum" : 1}}
#                                                       },
#                                                       {"$sort" : {"count" : -1}}
#                                                      ])

In [13]:
# for doc in amenities_counts:
#     print doc

In [14]:
buildings = db.city_data.find({"type":"node", "building":{"$exists":1}})

## Step 4: Speculate on Data Uses

<b>Amenity/Cuisine Densities</b>

The positions that are saved for each "node" document can be used to look at not only how popular a given amenity/cuisine type is, but how densely populated certain areas of the city are by those amenities. With that type of information I could compare two cities and look at a potential proxy for obesity by looking at the densities of fast food establishments between the two cities.

Another comparative analysis would involve the densities of police stations, hospitals, pharmacies, fire houses, and clinics per capita. Comparing those densities to somewhere like Auburn, Alabama or Seattle, WA could tell me how relatively well-equipped the cities are for emergencies.

<b>Accessibility/Traffic Flow</b>

By using the speed limit data from the "way" documents, iwe should be able to assess the objectively fastest route between two points, given no traffic and no traffic-regulation devices. Unfortunately, the data from Austin, TX doesn't include information about traffic signals on the roadways (lights or signs), so we couldn't use this data to find the true fastest route. But, to put more realism toward the project, we could use data from the U.S. Census to estimate the working population, the fraction of that population that uses a private vehicle to travel, the fraction that use public transport to travel, speed limit infoormation, and the assumption that no people (or some small fraction of people) live near working centers in order to estimate what traffic flow should be like for given hours of the day. It would no doubt be a crude estimation though.

<b>Enhancing a Real Estate App</b>

When looking for a new place to live (or reasons to mark up your place when trying to sell), a good thing to know would be a location's proximity to things people might want. We could use this data set to say for a given house location:
- How close is the nearest amenity?
- Within a given radius of miles/km, how many amenities are there? Of specific types?
- How close is the nearest highway? Highway onramp/offramp?
- What types of food are available within a given radius?

## Step 5: Conclusion

We've been able to clean the data from the OpenStreetMaps pull for Austin, TX. It's not absolutely perfect (street names for example), but it's pretty well standardized and ready for use in whatever comes next.

## Producing the sample file

In [16]:
## Taken from the instructor notes
outfile = "austin_sample.osm"

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(outfile, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every 20th top level element
    for i, element in enumerate(get_element(infile)):
        if i % 20 == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')